Document Classification

1. Text pre-processing and exploration:

In [146]:
import os
import nltk
import sklearn
import pandas as pd
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from os import listdir
from os.path import isfile, join    
import sklearn.datasets as sk

In [147]:
# here you should put the directory path
path='D:\\documents\\users\\nogahm\\Downloads\\ohsumed-first-20000-docs\\testCorpus'

dirpath=path
trainDirs=[]
testDirs=[]
trainFiles=[]
testFiles=[]
# for table
Category=[]
FileName=[]
Path=[]
Text=[]
classNumOfDocs=[]
docsInfo=[] #[fileName,cleanText,class

stopWords=(stopwords.words('english'))
stopWords += ['.', ',', '(', ')','%',';',':']

In [148]:
# get train and test dirs
trainDirs=os.listdir(dirpath+'\\training')
testDirs=os.listdir(dirpath+'\\test')

In [149]:
# pre-process
def cleanText(text):
    # tokenize
    tokenized = word_tokenize(text)
    # remove stop words
    filtered_sentence = [w for w in tokenized if not w in stopWords]
    # stemming
    ps = PorterStemmer()
    for i in range(len(filtered_sentence)):
        filtered_sentence[i] = ps.stem(filtered_sentence[i])
    # print("aaa")
    return " ".join(filtered_sentence)


In [150]:
# get train and test dirs
for dir in trainDirs:
    for file in os.listdir(dirpath+'\\training\\'+dir):
        trainFiles.append(file)
        Category.append(dir)
        FileName.append(file)
        Path.append(dirpath+'\\training\\'+dir+'\\'+file)
        # Open a file: fileReader
        fileReader = open(dirpath+'\\training\\'+dir+'\\'+file,mode='r')
        # read all lines at once
        text = fileReader.read().lower()
        # close the file
        fileReader.close()

        cleanedText=cleanText(text)
        docsInfo.append([file,cleanedText,dir,'training'])

for dir in testDirs:
    for file in os.listdir(dirpath+'\\test\\'+dir):
        testFiles.append(file)
        Category.append(dir)
        FileName.append(file)
        Path.append(dirpath+'\\test\\'+dir+'\\'+file)
        # Open a file: fileReader
        fileReader = open(dirpath+'\\test\\'+dir+'\\'+file,mode='r')
        # read all lines at once
        text = fileReader.read().lower()
        # close the file
        fileReader.close()

        cleanedText = cleanText(text)
        docsInfo.append([file, cleanedText, dir, 'test'])

In [141]:
#get number of classes and number of docs for each class
docInfoDF=pd.DataFrame(docsInfo, columns=['fileName','cleanText','class','group'])
allClasses=np.unique(docInfoDF['class'])
print ('# of categories: ',allClasses.size)
numOfFiles=[]
words_per_class = []
termsFreqPerClass=[]
for currClass in allClasses:
    temp=(docInfoDF['class']).value_counts()[currClass]
    numOfFiles.append([currClass, temp])
    #get terms distibution
    currFiles=docInfoDF.loc[docInfoDF['class']==currClass]
    termsFreq=pd.Series(" ".join(currFiles['cleanText']).split()).value_counts()[:10]
    # termsFreqPerClass.append([currClass,zip(termsFreq.index,termsFreq)])
    zipedFreq=zip(termsFreq.index, termsFreq)
    zippedList=list(zipedFreq)
    termsFreqPerClass.append([currClass]+zippedList)

# print number of files prt category
numOfFilesDF=pd.DataFrame(numOfFiles,columns=['Class','#OfFiles'])
print(numOfFilesDF)
# print terms frequency
pd.DataFrame(termsFreqPerClass, columns=['Class','term 1','term 2','term 3','term 4','term 5','term 6','term 7','term 8','term 9','term 10'])

# of categories:  4


  Class  #OfFiles
0   C01       929
1   C02       391
2   C03       135
3   C04      2630


,Class,term 1,term 2,term 3,term 4,term 5,term 6,term 7,term 8,term 9,term 10
0,C01,"(patient, 2145)","(infect, 1457)","(case, 551)","(group, 524)","(diseas, 516)","(studi, 513)","(treatment, 494)","(use, 464)","(less, 407)","(clinic, 407)"
1,C02,"(patient, 794)","(infect, 765)","(viru, 457)","(human, 348)","(hiv, 285)","(cell, 278)","(studi, 273)","(immunodefici, 264)","(diseas, 243)","(case, 240)"
2,C03,"(infect, 193)","(patient, 188)","(malaria, 118)","(antibodi, 84)","(case, 81)","(parasit, 80)","(respons, 73)","(studi, 73)","(day, 72)","(treatment, 72)"
3,C04,"(patient, 6719)","(cell, 3681)","(tumor, 3340)","(cancer, 2452)","(carcinoma, 1957)","(case, 1658)","(studi, 1655)","(use, 1493)","(treatment, 1308)","(diseas, 1270)"


<h1> 2. Document classification: <h1>

feature extraction methods: TF-IDF, Bag Of Words
machine learning models: SVM, Naïve Bayes

In [151]:
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Perceptron

In [158]:
# save best
best_accuracy = 0.0
best_fe = ''
best_classifier = ''
best_corpus_type = ''

In [159]:
train=docInfoDF.loc[docInfoDF['group']=='training']
trainSetText=train['cleanText'].tolist()
trainSetClass=train['class'].tolist()
test=docInfoDF.loc[docInfoDF['group']=='test']
testSetText=train['cleanText'].tolist()
testSetClass=train['class'].tolist()

In [164]:
def classify(extractM,machineLearningM ): 
            pipeline = Pipeline([('vect', extractM), ('clf', machineLearningM)])
            parameters = {'vect__max_df': np.arange(0.05, 0.55, 0.05),'clf__alpha': np.arange(0.01, 0.11, 0.01)}
            gs_clf = GridSearchCV(pipeline, parameters, n_jobs=1)
            gs_clf = gs_clf.fit(trainSetText, trainSetClass)
            prediction = gs_clf.predict(testSetText)
            accuracy = metrics.accuracy_score(testSetClass, prediction)
            print(accuracy)
            return accuracy

In [168]:
results=[]
#TF-IDF & SVM
print('TF-IDF AND SVM')
accuracy = classify(TfidfVectorizer(),SGDClassifier() )
results.append(['TF-IDF' , 'SVM', accuracy])


TF-IDF AND SVM


0.642896627971


In [ ]:
#TF-IDF & Naïve Bayes
print('TF-IDF AND Naïve Bayes')
accuracy = classify(TfidfVectorizer(),MultinomialNB() )
results.append(['TF-IDF' , 'Naïve Bayes', accuracy])

In [ ]:
#Bag of Words & SVM
print('Bag of Words AND SVM')
accuracy = classify(CountVectorizer(),SGDClassifier() )
results.append(['Bag of Words' , 'SVM', accuracy])

In [ ]:
#Bag of Words & Naïve Bayes
print('Bag of Words AND Naïve Bayes')
accuracy = classify(CountVectorizer(),MultinomialNB() )
results.append(['Bag of Words' , 'Naïve Bayes', accuracy])


In [ ]:
df.DataFrame(results,columns=['extraction method','machine learning method', 'accuracy'])